In [1]:
# libraries
import pandas as pd
import numpy as np
import requests

years = list(np.arange(1980,2006,1))
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
df = pd.DataFrame()
for y in years:
    for m in months:
        
        basketball = 'https://basketball.realgm.com/ncaa/birth-dates/' + str(y) + m + '01/3/'
        try:
            response = requests.get(basketball).text.replace('<!--', '').replace('-->', '')
            new_df = pd.read_html(response)[0]
            if not df.empty:
                df = pd.concat([df, new_df], axis=0)
            else:
                df = new_df
        except ValueError as ve:
            print(y, m, 'have no table.')
        
df

2005 12 have no table.


,Player,Pos,HT,WT,Birth City,Birth Date,Nationality,College,Current Team
0,Damien Wilkins,GF,6-6,225,"Washington, NC","Jan 11, 1980",United States,Georgia,-
1,Omar Barlett,F,6-8,234,"Miami, FL","Jan 20, 1980",United States,Jacksonville State,-
2,Aaron Pettway,PF,6-10,240,"Mobile, AL","Jan 4, 1980",United States,Oklahoma State,-
3,Hector Romero,PF,6-8,240,Barcelona,"Jan 3, 1980",Venezuela,New Orleans,-
4,Doug Wrenn,F,6-6,215,"Seattle, WA","Jan 21, 1980",United States,Washington,-
...,...,...,...,...,...,...,...,...,...
0,Myles Colvin,SG,6-5,210,"Indianapolis, IN","Aug 9, 2005",United States,Purdue,Purdue
0,"Jimmy Oladokun, Jr.",PF,6-9,220,"La Verne, CA","Sep 19, 2005",United States,San Diego,San Diego
0,Jacob Theodosiou,G,6-4,188,Waterloo,"Oct 4, 2005",Canada,Wyoming,Wyoming
1,Jarin Stevenson,F,6-11,210,"Fayetteville, NC","Oct 15, 2005",United States,Alabama,Alabama


In [2]:
import calendar

df['birth_month'] = df['Birth Date'].str.strip().str.split(' ').str[0]
df_new = df.drop(['Pos', 'HT', 'WT', 'Birth City', 'Nationality', 'College', 'Current Team', 'Birth Date'], axis=1)

lower_ma = [m.lower() for m in calendar.month_abbr]

df_new['birth_month'] = df_new['birth_month'].str.lower().map(lambda m: lower_ma.index(m)).astype('Int8')

month = df_new[['birth_month','Player']].groupby(['birth_month']).count().reset_index().rename(columns={"Player": "Count"})
month['birth_month'] = month['birth_month'].apply(lambda x: calendar.month_abbr[x])

correction = [31, 28.25, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
c_month = month.copy()

count=0
for index, row in month.iterrows():
    c_month.loc[index, 'Count'] *= 30/correction[count]
    count +=1

c_month.Count = c_month.Count.round(2)
c_month

,birth_month,Count
0,Jan,3092.90
1,Feb,2902.30
2,Mar,2971.94
3,Apr,2835.00
4,May,2776.45
5,Jun,2759.00
6,Jul,2806.45
7,Aug,2831.61
8,Sep,3246.00
9,Oct,3022.26


In [5]:
c_month.to_csv('ncaa_bb.csv')